carga los puntos de recorridos de surcado a base de datos

In [109]:
import sys
sys.path.append('..')

from sqlalchemy import create_engine

import geopandas as gpd
import os
import pandas as pd
from pathlib import Path

In [110]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

In [111]:
POSTGRES_UTEA['DATABASE'] = 'utea_precision'

In [127]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def existe_puntos_de_recorrdido(unidad_01, unidad_05, anho):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT COUNT(*) total FROM siembra_surcado.puntos_recorrido_surcado 
            WHERE unidad_01 = {unidad_01} AND unidad_05 = '{unidad_05}' AND anho = {anho}
        """
        df = pd.read_sql(query, engine)
        cantidad = df['total'].iloc[0] # Retorna directamente el número de registros
        if cantidad > 0:
            return True
        else:
            return False
    except Exception as e:
        print(f"❌ Error en la consulta existe_puntos_de_recorrdido: {e}")
        return False
    return None

def eliminar_puntos_de_recorrido(unidad_01, unidad_05, anho):
    engine = obtener_engine()
    # Usamos parámetros (:u1, etc.) para que la consulta sea segura y limpia
    query = text("""
        DELETE FROM siembra_surcado.puntos_recorrido_surcado
        WHERE unidad_01 = :u1 AND unidad_05 = :u5 AND anho = :a
    """)
    try:
        with engine.connect() as conn:
            # Ejecutamos la sentencia
            result = conn.execute(query, {"u1": unidad_01, "u5": unidad_05, "a": anho})
            # ¡Muy importante! Para que los cambios se guarden permanentemente
            conn.commit() 
            # result.rowcount nos dice cuántas filas fueron eliminadas
            if result.rowcount > 0:
                print(f"✅ Se eliminaron {result.rowcount} puntos antiguos.")
                return True
            else:
                print("ℹ️ No se encontraron puntos para eliminar.")
                return False
    except Exception as e:
        print(f"❌ Error al ejecutar eliminar_puntos_de_recorrido(): {e}")
        return False

In [113]:
# ruta de shps
ruta_shp = r"C:/Documents/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - SIEMBRA/SIEMBRA CON PRESICION/2026/SHP_PUNTOS"
# carpeta path
carpeta = Path(ruta)
# filtra el contenido solo de .shps
contenido_shp = [f.name for f in carpeta.glob("*.shp")]
contenido_shp

['2238_EL PARAISO_L5_2026.shp',
 '30_CAMPO DULCE_EP-L33-EP-L34_2026.shp',
 '30_CAMPO DULCE_EP-L9_2026.shp']

In [114]:
# extrae el primer shp de la lista
archivo = contenido_shp[0]
# split para quitar la extencion
nombre = archivo.split('.')[0]
# split para separar unidades y anho
unidad_01, unidad_02, unidad_05, anho = nombre.split('_')

In [128]:
existe_puntos = existe_puntos_de_recorrdido(unidad_01, unidad_05, anho)

In [136]:
if existe_puntos == True:
    eliminar_puntos_de_recorrido(unidad_01, unidad_05, anho)

In [137]:
# concatena la ruta de la carpeta y el nombre del archivo
ruta_completa = os.path.join(carpeta, archivo)
# importa el shp en geodataframe
gdf_puntos = gpd.read_file(ruta_completa)
# extraer las columnas requeridas
gdf_puntos = gdf_puntos[['Time', 'Heading', 'DISTANCE', 'SWATHWIDTH', 'Elevation', 'IsoTime', 'geometry']]
# asignar valores de unidades y anho
gdf_puntos['unidad_01'] = unidad_01
gdf_puntos['unidad_02'] = unidad_02
gdf_puntos['unidad_05'] = unidad_05
gdf_puntos['anho'] = anho
# convertir a datatime la columna isotime
gdf_puntos['IsoTime'] = pd.to_datetime(gdf_puntos['IsoTime'])
# copiar en columna anho el año de isotime
# gdf_puntos['anho'] = gdf_puntos['IsoTime'].dt.year
# convertir todos los numbres de columna a minusculas
gdf_puntos.columns = gdf_puntos.columns.str.lower()
# idd que convina unidad_01, unidad_05, y anho
gdf_puntos['idd'] = (
    gdf_puntos['unidad_01'].astype(str) + "|" + 
    gdf_puntos['unidad_05'].astype(str) + "|" + 
    gdf_puntos['anho'].astype(str)
)
# renombre de la columna geometria
gdf_puntos = gdf_puntos.rename(columns={'geometry': 'geom'})
# reasignancion de la columna geometria
gdf_puntos = gdf_puntos.set_geometry('geom')
# Transformar a la zona 20S (metros)
gdf_puntos = gdf_puntos.to_crs(epsg=32720)

In [138]:
gdf_puntos.to_postgis("puntos_recorrido_surcado", obtener_engine(), schema="siembra_surcado", if_exists="append")